# `true-voice` Classification using ULMfit

Reference: https://github.com/cstorm125/thai2fit
and https://docs.fast.ai/

First step: create a folder called "ulmfit-lab" in your google drive, then copy these files https://drive.google.com/drive/folders/1b9vTH4qpXKNxxT2F3qaspqWfFkJ4f9Vg?usp=share_link
 to the folder.

In [ ]:
#Don't for get to restart before doing your works.
!pip install fastai==1.0.46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 42.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19188 sha256=511b43c2a863bd145c4f15d471eb3d50b4dcd5cdfaff5fde48a041b647093b95
  Stored in directory: /root/.cache/pip/wheels/f6/d8/b0/15cfd7805d39250ac29318105f09b1750683387630d68423e1
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26321 sha256=13f994867f1d12578b4f6f6660b216d3e6d537b58253633bca3c6aa32a43f71a
  Stored in directory: /root/.cache/pip/wheels/fa/17/1f/332799f975d1b2d7f9b3f33bbccf65031e794717d24432caee
Successfully built nvidia-ml-py3 typing
  At

In [ ]:
!pip install pythainlp[thai2fit]
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=c2287227df118cd039331e7e0b9b3a99ab61a50c66520aadfdc4bae1d18ffdee
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'FRA 501 IntroNLP&DL/Dataset/ulmfit-lab/'

Mounted at /content/gdrive


In [ ]:
base_dir

'/content/gdrive/My Drive/FRA 501 IntroNLP&DL/Dataset/ulmfit-lab/'

In [ ]:
import shutil 
shutil.copy("/content/gdrive/MyDrive/FRA 501 IntroNLP&DL/Dataset/clean-phone-data.csv", "/content/clean-phone-data.csv")
!head clean-phone-data.csv

Sentence Utterance,Action,Object
 <PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท,enquire,payment
 internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
 ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ,report,suspend
 พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง โกลไล,enquire,internet
 ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโทรออกไม่ได้คะ แต่เล่นเนตได้คะ,report,phone_issues
*2222 ใช้งานยังไง ขอรายละเอียดการสมัครหน่อย,enquire,service
<PHONE_NUMBER_REMOVED> เคยมีช่างมาซ่อมที่บ้าน แล้วโทรศัพท์ใช้งานไม่ได้ครับ,enquire,nonTrueMove
<PHONE_NUMBER_REMOVED> ค้างค่าบริการเท่าไหร่ครับ,enquire,balance
<PHONE_NUMBER_REMOVED> อินเตอร์เน็ตไฟ Adsl ไม่มีสัญญาณครับ,enquire,nonTrueMove


In [ ]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import pythainlp

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

##Load data

First, we load the data from disk into a Dataframe.
A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.
**Note: the data is provided courtesy of [TrueVoice](http://www.truevoice.co.th/) and must be used for educational purposes only**

In [ ]:
phone_df = pd.read_csv('clean-phone-data.csv')
# Show the top 5 rows
display(phone_df.head())
# Summarize the data
phone_df.describe()


,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We first select only the columns we are interested.

For this example we will be training a model to predict the "Object" given a "Sentence Utterance". Thus we will be only selecting these two. Others can also be selected, but these two will serve as an example.

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

In [ ]:
data_df = phone_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

### Removing duplicated labels

from above, we can see that there are 33 unique labels

But we want to make sure that these label are really unique from one another not just how capital letters are used

In [ ]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
display(data_df.describe())
display(data_df.clean_label.unique())

,input,raw_label,clean_label
count,16175,16175,16175
unique,13389,33,26
top,บริการอื่นๆ,service,service
freq,97,2525,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [ ]:
#we no longer need raw_label column
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


### Remove duplicates input

There are some duplicates in the input of this dataset.

In [ ]:
display(data_df[data_df.duplicated(keep=False)]['input'])

18       <PHONE_NUMBER_REMOVED> เป็นหมายเลขของเครือข่าย...
19       <PHONE_NUMBER_REMOVED> เป็นหมายเลขของเครือข่าย...
35       <PHONE_NUMBER_REMOVED> เป็นเติมเงินนะคะ เดี๋ยว...
36       <PHONE_NUMBER_REMOVED> เป็นเติมเงินนะคะ เดี๋ยว...
52               3g wifi edge ตอนนี้มีโปรโมชั่นอะไรบ้างค่ะ
                               ...                        
16165                                         ดาต้าโรมมิ่ง
16166                                สอบถามยอดค่าบริการค่ะ
16168                          ใช้งานอินเตอร์เน็ตไม่ได้ค่ะ
16169                          ใช้งานอินเตอร์เน็ตไม่ได้ค่ะ
16171                                  โทรออกต่างประเทศค่ะ
Name: input, Length: 4376, dtype: object

As you can see above, we no longer have unwanted duplicated in our labels

But there are still duplicates in our input. 7 rows with "สอบถามยอดค้างชำระค่ะ"

We will remove them now, by keeping only the first entry.

In [ ]:
data_df = data_df.drop_duplicates("input", keep="first")
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,service
freq,1,2111


### Trim whitestring

In [ ]:
data_df['input'] = data_df['input'].apply(lambda x: x.strip() )
unique_label = data_df.clean_label.unique()

### Split data

split data into train and test set 

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data_df,stratify=data_df['clean_label'], test_size=0.2,random_state=42)


In [ ]:
train_data.head()

,input,clean_label
8833,พอดีจะเชื่อมต่อ wifi ผมเปลี่ยนเครื่องครับ,internet
9965,มีเรื่องจะสอบเกี่ยว wifi แรงไม่อั้นสูงสุด 8 MB...,promotion
8426,ผมสอบถามข้อมูลเน็ตทรูออนไลน์หน่อยครับตรงพื้นที...,information
7153,ตอนนี้อยู่ที่อยุธยา ใช้ 3G เอจ ไม่ได้เลยคะ เจ้...,phone_issues
2111,ข้อความครับ,nontruemove


In [ ]:
len(train_data)

10711

## [ULMFit](https://arxiv.org/abs/1801.06146) Model
ULMFit model has 3 stages:
1. The LM is trained on a general-domain corpus to capture
general features of the language in different layers
2. The full LM is fine-tuned on target task data 
3. The classifier is fine-tuned on the target task using gradual unfreezing, ‘Discr’, and STLR to
preserve low-level representations and adapt high-level ones 

In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
from tqdm import tqdm_notebook
from collections import Counter
import re

#viz
import matplotlib.pyplot as plt
import seaborn as sns

from fastai.text import * #ThaiTokenizer
from fastai import *
from fastai.callbacks import CSVLogger, SaveModelCallback

from pythainlp.ulmfit import *

model_path = ''

Corpus: wiki_lm_lstm
- Downloading: wiki_lm_lstm 0.32


  0%|          | 0/1050919089 [00:00<?, ?it/s]

Corpus: wiki_itos_lstm
- Downloading: wiki_itos_lstm 0.32


  0%|          | 0/1530484 [00:00<?, ?it/s]

In [ ]:
THWIKI_LSTM #The LM is trained on a general-domain corpus to capture general features of the language in different layers

{'wgts_fname': '/root/pythainlp-data/thwiki_model_lstm.pth',
 'itos_fname': '/root/pythainlp-data/itos_lstm.pkl'}

In [ ]:
#configuration
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=None, max_vocab=60000, min_freq=3)]

data_lm = (TextList.from_df(train_data, model_path, cols=['input'], processor=processor)
    .random_split_by_pct(valid_pct = 0.01, seed = 1412)
    .label_for_lm()
    .databunch(bs=64))
data_lm.sanity_check()
data_lm.save('truevoice_FIBO_lm.pkl')

In [ ]:
data_lm = load_data(model_path,'truevoice_FIBO_lm.pkl') #Target task data
data_lm.sanity_check()

In [ ]:
len(data_lm.train_ds), len(data_lm.valid_ds)

(10604, 107)

In [ ]:
data_lm.show_batch(5)

idx,text
0,xxrep 5 คือ คือ ใช้ รายเดือน เท่าไร ครับ xxbos จะ โทร ต่างประเทศ ต้อง ทำ ยังไง xxbos จ่าย ค่า โทร ไป แล้ว วันนี้ นาน ไหม คับ กว่า จะ กลับมา โทร ได้ อีก xxbos อยาก ใช้งาน wifi 100 บาท แต่ ไม่มี เบอร์ truemove xxbos ขอ เปลี่ยน ที่อยู่ ที่แจ้ง
1,จะ สอบถาม เรื่อง โทรศัพท์ มัน โทร เข้า โทร ออก ไม่ได้ ครับ xxbos ระงับ การ ใช้ โทรศัพท์ เนื่องจาก เครื่อง ถูก ขโมย xxbos อยาก ทราบ ว่าที่ หัก ค่า โทร 1800 รวม ค่า อินเตอร์เนต มั้ย ค่ะ xxbos จะ เพิ่ม วงเงิน ครับ xxbos สมัคร แพ็คเกจ อินเตอร์เนต รายวัน คะ xxbos ขาย บัตร ท รู มัน นี่ ได้
2,หาก ท รู มูฟ ใช้ โปรโมชั่น โทร หา ท รู มูฟ ด้วยกัน ฟรี แต่ ถ้า โทร หา ท รู มูฟ ฟรี หรือไม่ xxbos ท รู มูฟ เอ ช แบบ เติม เงิน เล่น บีบี แชท เดือน ละ เท่าไหร่ xxbos
3,สอบถาม ว่า รอบ บิล วันที่ 25 / 3 / 55 - 9 / 4 / 55 และ วันที่ 10 / 4 / 55
4,สอบถาม ว่า มี ข้อความ ส่ง มา ว่า หมายเลข ของ ผม อะไร ที่อยู่ ซัก ที่ ได้ ส่ง ไป ไปรษณีย์ คือ อะไร ครับ เอ่อ ใบ แจ้ง ค่าบริการ ท รู มูฟ เอ ช ของ ท่าน หมายเลข นี้ ได้รับ การ ตี คืน จาก ไปรษณีย์ xxbos เช็ค ยอด ให้ พี่ หน่อย ค่ะ เท่าไหร่ ค่ะ xxbos ขอ สอบถาม นิดนึง เรื่อง


Meaning of special Token

* UNK (xxunk) is for an unknown word (one that isn't present in the current vocabulary)
* PAD (xxpad) is the token used for padding, if we need to regroup several texts of different lengths in a batch
* BOS (xxbos) represents the beginning of a text in your dataset
* FLD (xxfld) is used if you set mark_fields=True in your TokenizeProcessor to separate the different fields of texts (if your texts are loaded from several columns in a dataframe)
* TK_MAJ (xxmaj) is used to indicate the next word begins with a capital in the original text
* TK_UP (xxup) is used to indicate the next word is written in all caps in the original text
* TK_REP (xxrep) is used to indicate the next character is repeated n times in the original text (usage xxrep n {char})
* TK_WREP(xxwrep) is used to indicate the next word is repeated n times in the original text (usage xxwrep n {word})

Credit https://docs.fast.ai/text.transform.html

### (step 1) load pretrained model trained on general corpus

Create language_model_learner with AWD-LSTM as RNN, LSTM model we will explain later.


AWD_LSTM is a language model trained on a large corpus from a section of Wikipedia called WikiText-103.
It has already reached a size of 103,227,021 Tokens and 267,735 Vocab.

In [ ]:
#configuration
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False, tie_weights=True, out_bias=True,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(drop_mult=0.9, clip=0.12, alpha=2, beta=1)

learn = language_model_learner(data_lm, AWD_LSTM, config=config, pretrained=False, **trn_args) #for step 2

# load pretrained models ( Step 1. The LM is trained on a general-domain corpus to capture
# general features of the language in different layers)
learn.load_pretrained(**THWIKI_LSTM) 

### (step 2) the full LM is fine-tuned on target task data 

Read about freeze_to

In [ ]:
#train frozen
print('training frozen')
learn.freeze_to(-1)
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7))

training frozen


epoch,train_loss,valid_loss,accuracy,time
1,6.153827,5.690939,0.212946,00:14


unfreeze sets every layer group to trainable 

In [ ]:
#train unfrozen (take about 5 mins)
print('training unfrozen')
learn.unfreeze()
learn.fit_one_cycle(10, 1e-4, moms=(0.8, 0.7))

training unfrozen


epoch,train_loss,valid_loss,accuracy,time
1,5.698646,5.128852,0.223661,00:20
2,5.181168,4.326212,0.273884,00:21
3,4.651645,3.912117,0.307813,00:20
4,4.287653,3.721354,0.327455,00:20
5,4.044956,3.616833,0.341071,00:20
6,3.886734,3.557289,0.345982,00:20
7,3.773396,3.522802,0.350670,00:20
8,3.703629,3.504820,0.345759,00:20
9,3.666207,3.498927,0.352232,00:20
10,3.648154,3.499959,0.350223,00:20


In [ ]:
# you can skip the above step by loading the pretrained weight instead
# learn.load(base_dir+'truevoice_lm')
# learn.unfreeze()
# learn.fit_one_cycle(1, 1e-4, moms=(0.8, 0.7))

In [ ]:
learn.save_encoder('truevoice_enc')

### (step 3) Classification
 The classifier is fine-tuned on the target task using gradual unfreezing, ‘Discr’, and STLR to
preserve low-level representations and adapt high-level ones 

In [ ]:
#configuration
tt = Tokenizer(tok_func = ThaiTokenizer, lang = 'th', pre_rules = pre_rules_th, post_rules=post_rules_th)
processor = [TokenizeProcessor(tokenizer=tt, chunksize=10000, mark_fields=False),
            NumericalizeProcessor(vocab=data_lm.vocab, max_vocab=60000, min_freq=3)]


data_cls = (TextList.from_df(train_data, model_path, cols=['input'], processor=processor)
    .random_split_by_pct(valid_pct = 0.05, seed = 1412)
    .label_from_df('clean_label')
    .add_test(TextList.from_df(test_data, model_path, cols=['input'], processor=processor))
    .databunch(bs=64)
    )

data_cls.sanity_check()
data_cls.save('truevoice_FIBO_cls.pkl')

In [ ]:
config = dict(emb_sz=400, n_hid=1550, n_layers=4, pad_token=1, qrnn=False,
             output_p=0.25, hidden_p=0.1, input_p=0.2, embed_p=0.02, weight_p=0.15)
trn_args = dict(bptt=70, drop_mult=0.5, alpha=2, beta=1)

learn = text_classifier_learner(data_cls, AWD_LSTM, config=config, pretrained=False, **trn_args)
learn.opt_func = partial(optim.Adam, betas=(0.7, 0.99))

#load pretrained finetuned model
learn.load_encoder('truevoice_enc')

In [ ]:
#train unfrozen
learn.freeze_to(-1)
learn.fit_one_cycle(1, 2e-2, moms=(0.8, 0.7))

epoch,train_loss,valid_loss,accuracy,time
1,1.628413,1.422137,0.571963,00:11


In [ ]:
#gradual unfreezing (5 mins)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2 / (2.6 ** 4), 1e-2), moms=(0.8, 0.7))
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3 / (2.6 ** 4), 5e-3), moms=(0.8, 0.7))
learn.unfreeze()
learn.fit_one_cycle(4, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7), 
                   callbacks=[SaveModelCallback(learn, every='improvement', monitor='accuracy', name='truevoice_cls')])

epoch,train_loss,valid_loss,accuracy,time
1,1.247957,1.170256,0.657944,00:12


epoch,train_loss,valid_loss,accuracy,time
1,0.982656,1.014654,0.697196,00:20


epoch,train_loss,valid_loss,accuracy,time
1,0.729241,0.987865,0.699065,00:31
2,0.651093,0.957066,0.714019,00:32
3,0.436133,0.974202,0.723364,00:32
4,0.355838,0.978816,0.721495,00:32


Better model found at epoch 1 with accuracy value: 0.6990654468536377.
Better model found at epoch 2 with accuracy value: 0.714018702507019.
Better model found at epoch 3 with accuracy value: 0.7233644723892212.


In [ ]:
# To skip the above step, run this cell instead
# learn.load(base_dir+'final')
# learn.fit_one_cycle(1, slice(1e-3 / (2.6 ** 4), 1e-3), moms=(0.8, 0.7))

# Evaluation

In [ ]:
probs, _ = learn.get_preds(ds_type = DatasetType.Test, ordered=True)
classes = learn.data.train_ds.classes
preds = np.array([classes[i] for i in probs.argmax(1).numpy()])
prob = probs.numpy()

In [ ]:
print("ULMFit Model Acc: %f%%" % ((test_data['clean_label']== preds).sum() / test_data.shape[0] * 100))
print("ULMFit Model Micro f1: %f%%" % f1_score(test_data['clean_label'], preds, average='micro') )
print("ULMFit Model Macro f1: %f%%" % f1_score(test_data['clean_label'], preds, average='macro') )

ULMFit Model Acc: 73.637043%
ULMFit Model Micro f1: 0.736370%
ULMFit Model Macro f1: 0.562627%
